In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nih-chest-xrays/data")

print("Path to dataset files:", path)

In [ ]:
import tensorflow as tf
print("GPUs available:", tf.config.list_physical_devices('GPU'))


In [ ]:
# ============================================================
# CELL 1 — IMPORTS & ENVIRONMENT SETUP
# ============================================================

import os, glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

print("TensorFlow version:", tf._version_)
print("Available GPUs:", tf.config.list_physical_devices("GPU"))

# ---- Stability & Reproducibility ----
# Disable mixed precision & XLA for numerical stability
tf.keras.mixed_precision.set_global_policy("float32")
tf.config.optimizer.set_jit(False)

SEED = 42
tf.keras.utils.set_random_seed(SEED)
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# ============================================================
# CELL 2 — GLOBAL CONFIGURATION
# ============================================================

IMG_SIZE = (224, 224)     # Standard size for medical CNNs
BATCH    = 32             # Balanced for GPU memory + stability
EPOCHS  = 20

In [ ]:
# ============================================================
# CELL 3 — LOCATE DATASET FILES
# ============================================================

ROOT = "/kaggle/input"

# Locate CSV metadata file
csv_files = glob.glob(os.path.join(ROOT, "**", "Data_Entry_2017.csv"), recursive=True)
if not csv_files:
    raise FileNotFoundError("Data_Entry_2017.csv not found")

CSV_PATH = csv_files[0]
print("CSV file found at:", CSV_PATH)

# Locate all image files
png_files = glob.glob(os.path.join(ROOT, "*", ".png"), recursive=True)
print("Total PNG images found:", len(png_files))

# Map filename → full path
fname_to_path = {os.path.basename(p): p for p in png_files}

In [ ]:
# ============================================================
# CELL 4 — LOAD CSV & CREATE BINARY LABEL
# ============================================================

df = pd.read_csv(CSV_PATH)

# Keep only images that exist on disk
df = df[df["Image Index"].isin(fname_to_path)]
df["path"] = df["Image Index"].map(fname_to_path)

# ---- Binary Label Definition ----
# 1 → Any disease present
# 0 → "No Finding"
df["label"] = (df["Finding Labels"] != "No Finding").astype(np.float32)

print("Total images used:", len(df))
print("Label distribution:")
print(df["label"].value_counts())
print("Positive class ratio:", df["label"].mean())

In [ ]:
# ============================================================
# CELL 5 — PATIENT-WISE TRAIN / VAL / TEST SPLIT
# ============================================================

# IMPORTANT:
# We split by Patient ID to avoid data leakage

patients = df["Patient ID"].unique()
np.random.shuffle(patients)

n = len(patients)
train_p = set(patients[:int(0.70 * n)])
val_p   = set(patients[int(0.70 * n):int(0.85 * n)])
test_p  = set(patients[int(0.85 * n):])

train_df = df[df["Patient ID"].isin(train_p)]
val_df   = df[df["Patient ID"].isin(val_p)]
test_df  = df[df["Patient ID"].isin(test_p)]

print("Train samples:", len(train_df))
print("Validation samples:", len(val_df))
print("Test samples:", len(test_df))

In [ ]:
# ============================================================
# CELL 6 — IMAGE LOADING (NO AUGMENTATION)
# ============================================================

def decode_img(path):
    """
    Reads PNG image, converts to grayscale,
    resizes, and normalizes to [0,1]
    """
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=1)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return img

def make_ds(df_part, training):
    """
    Creates tf.data pipeline
    NO DATA AUGMENTATION (as per professor’s instruction)
    """
    ds = tf.data.Dataset.from_tensor_slices(
        (df_part["path"].values, df_part["label"].values)
    )

    # Shuffle only (not augmentation)
    if training:
        ds = ds.shuffle(8192, seed=SEED, reshuffle_each_iteration=True)

    def _map(p, y):
        img = decode_img(p)
        return img, tf.reshape(tf.cast(y, tf.float32), (1,))

    ds = ds.map(_map, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH).prefetch(AUTOTUNE)
    return ds

train_ds = make_ds(train_df, True)
val_ds   = make_ds(val_df, False)
test_ds  = make_ds(test_df, False)

In [ ]:
# ============================================================
# CELL 7 — CBAM ATTENTION BLOCK (SAFE KERAS VERSION)
# ============================================================

def cbam_block(x, ratio=8):
    channel = x.shape[-1]

    # ----- Channel Attention -----
    avg_pool = layers.GlobalAveragePooling2D()(x)
    max_pool = layers.GlobalMaxPooling2D()(x)

    shared_dense_1 = layers.Dense(channel // ratio, activation="relu", use_bias=False)
    shared_dense_2 = layers.Dense(channel, use_bias=False)

    avg_out = shared_dense_2(shared_dense_1(avg_pool))
    max_out = shared_dense_2(shared_dense_1(max_pool))

    ch_att = layers.Activation("sigmoid")(layers.Add()([avg_out, max_out]))
    ch_att = layers.Reshape((1, 1, channel))(ch_att)
    x = layers.Multiply()([x, ch_att])

    # ----- Spatial Attention (NO Lambda) -----
    avg_sp = layers.AveragePooling2D(pool_size=(1, 1))(x)
    max_sp = layers.MaxPooling2D(pool_size=(1, 1))(x)

    sp = layers.Concatenate(axis=-1)([avg_sp, max_sp])
    sp = layers.Conv2D(1, 7, padding="same", activation="sigmoid")(sp)

    return layers.Multiply()([x, sp])

In [ ]:
# ============================================================
# CELL 8 — ATTENTION GATE (FROM ATTENTION U-NET)
# ============================================================

def attention_gate(skip, gating, inter):
    gating = layers.Resizing(skip.shape[1], skip.shape[2])(gating)

    theta = layers.Conv2D(inter, 1)(skip)
    phi   = layers.Conv2D(inter, 1)(gating)

    x = layers.Activation("relu")(layers.Add()([theta, phi]))
    psi = layers.Conv2D(1, 1, activation="sigmoid")(x)

    return layers.Multiply()([skip, psi])

In [ ]:
# ============================================================
# CELL 9 — MODEL ARCHITECTURE (CBAM + ATTENTION U-NET)
# ============================================================

def conv_block(x, f):
    x = layers.Conv2D(f, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("swish")(x)

    x = layers.Conv2D(f, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    return layers.Activation("swish")(x)

def enc(x, f):
    s = conv_block(x, f)
    s = cbam_block(s)
    p = layers.MaxPooling2D()(s)
    return s, p

def build_model():
    inp = layers.Input((*IMG_SIZE, 1))

    c1,p1 = enc(inp,32)
    c2,p2 = enc(p1,64)
    c3,p3 = enc(p2,128)
    c4,p4 = enc(p3,256)

    b = conv_block(p4,512)

    a4 = attention_gate(c4,b,128)
    u4 = conv_block(layers.Concatenate()([layers.UpSampling2D()(b),a4]),256)

    a3 = attention_gate(c3,u4,64)
    u3 = conv_block(layers.Concatenate()([layers.UpSampling2D()(u4),a3]),128)

    a2 = attention_gate(c2,u3,32)
    u2 = conv_block(layers.Concatenate()([layers.UpSampling2D()(u3),a2]),64)

    a1 = attention_gate(c1,u2,16)
    u1 = conv_block(layers.Concatenate()([layers.UpSampling2D()(u2),a1]),32)

    x = layers.GlobalAveragePooling2D()(u1)
    x = layers.Dense(128, activation="swish")(x)
    x = layers.Dropout(0.3)(x)

    out = layers.Dense(1, activation="sigmoid")(x)
    return models.Model(inp, out)

model = build_model()
model.summary()

In [ ]:
[16:26, 12/02/2026] Pranav Kerala Vit Friend Phy 2: # ============================================================
# CELL 10 — CUSTOM F1 METRIC
# ============================================================

def f1(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))
    p = tp / (tp + fp + 1e-7)
    r = tp / (tp + fn + 1e-7)
    return 2 * p * r / (p + r + 1e-7)


In [ ]:
# ============================================================
# CELL 11 — COMPILE MODEL
# ============================================================

model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.05),
    metrics=[
        keras.metrics.BinaryAccuracy(name="accuracy"),
        keras.metrics.AUC(name="auc"),
        keras.metrics.Precision(),
        keras.metrics.Recall(),
        f1
    ]
)

In [ ]:
# ============================================================
# CELL 12 — TRAINING
# ============================================================

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_auc", patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_auc", factor=0.5, patience=2)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
# ============================================================
# CELL 13 — THRESHOLD OPTIMIZATION (VALIDATION)
# ============================================================

y_true, y_prob = [], []

for x, y in val_ds:
    p = model.predict(x, verbose=0)
    y_true.append(y.numpy().ravel())
    y_prob.append(p.ravel())

y_true = np.concatenate(y_true)
y_prob = np.concatenate(y_prob)

best_acc, best_t = 0, 0.5
for t in np.linspace(0.1, 0.9, 81):
    acc = ((y_prob >= t) == y_true).mean()
    if acc > best_acc:
        best_acc, best_t = acc, t

print("Best Validation Accuracy:", best_acc)
print("Best Threshold:", best_t)

In [ ]:
# ============================================================
# CELL 14 — FINAL TEST EVALUATION
# ============================================================

model.evaluate(test_ds)